In [1]:
import pandas as pd
import numpy as np
import time
from src.peblMl import *

In [65]:
%%time
data_start=pd.read_pickle('data/wikiNews.pkl')
data_train=data_start[:10000]
data_test=data_start[10000:10100]
#data_test=data_test[~data_test['topic'].isin(['other'])]
#data_train=data_train[~data_train['topic'].isin(['other'])]
print (len(data_test), len(data_train))

100 10000
Wall time: 136 ms


In [66]:
data_train.head()

,Noun,Text,category,topic,date_news
7376,мефодий год жизнь поэт член союз писатель росс...,владимир мефодьевич сегодня из пришло известие...,барнаул алтайский_край некрологи писатели росс...,"россия, некрологи",2005-02-01
1349,россия премьерминистр украина разъяснение вним...,россия заявлениями премьерминистра украины хоч...,россия украина политика вто юрий_ехануров поли...,"россия, украина, политика, европа",2005-11-01
1516,спортиспанияс человек гроссмейстер программа ч...,матч люди компьютеров спортиспанияс по ноября ...,фиде,other,2005-11-01
1520,ангел меркель германия германия женщина правит...,ангела меркель канцлер германии германия она п...,ангела_меркель христианско_демократический_сою...,политика,2005-11-01
1527,атака ноябрь страница атака домен машина гугл ...,атака гугл данным компании ноября предпринята ...,google швейцария европа интернет хакерские_ата...,европа,2005-11-01


In [96]:
%%time
modelML=peblMl(factor=0.2)
modelML.fit(data_train['Noun'].tolist(), data_train['category'].tolist())

Wall time: 18.9 s


In [97]:
r_test_proverka=modelML.predict('afshdlfahlsjhfda lsdafhskdhfla здоровье здоровье здоровье', 'newTopic')[:10]

In [98]:
r_test_proverka

newTopic                                    0.117649
нефть                                       0.011864
резервный_фонд_российской_федерации         0.010381
военно_морской_флот_российской_федерации    0.008898
здоровье                                    0.008898
балтийское_море                             0.008898
байкал                                      0.008898
инновации                                   0.008898
медицина                                    0.008898
европа                                      0.008898
dtype: float64

In [99]:
print (modelML.penalty, modelML.topicNewTest.sum())

0.4 2.999949580679317


In [100]:
ar_avg_prob=modelML.ProbTopicWord

In [101]:
print (np.amax(ar_avg_prob, axis=1), np.average(np.amax(ar_avg_prob, axis=1)), np.median(np.amax(ar_avg_prob, axis=1)))

[0.19990005 0.24937656 0.49751244 ... 0.05878895 0.09082652 0.09086779] 0.3132781998350957 0.1998001998001998


In [102]:
print (len(np.amax(ar_avg_prob, axis=1)), len(np.amax(ar_avg_prob, axis=0)))

4057 86182


In [103]:
%%time
topicValueResult=pd.DataFrame(columns=['topicName', 'probValue', 'countNewWord'])
i=0
for testRow in data_test['Noun'].tolist():
    result=modelML.predict(testRow, 'newTopic')[:2]
    topicValueResult=topicValueResult.append({'topicName': result.index[0], 'probValue': result.values[0], 'countNewWord': modelML.penalty}, ignore_index=True)
    if i%10==0:
        print (i, result.index[0], result.values[0],  
                result.index[1], result.values[1], 
               data_test['category'][i], modelML.penalty)
    i+=1
    #if i%10==0:
    #    print (result.index[0], result.values[0])


0 newTopic 0.13336576209846174 джордж_уокер_буш 0.017618025177251473  2.0
10 newTopic 0.03942246070195591 олимпийские_игры 0.03705426644417842  1.6
20 ванкувер 0.0628290612397951 российско_американские_отношения 0.0541735890577583  0.2
30 newTopic 0.038461718061436896 россия 0.01704607916299757  0.2
40 newTopic 0.07146440570086943 женщины 0.05166921820014052  1.0
50 newTopic 0.017857734214122154 россия 0.017665325848857175  0.2
60 newTopic 0.023813784652762713 роскомнадзор 0.018976802823515115  1.0
70 политика 0.04963843973968544 нурсултан_назарбаев 0.04637252892186747  0.6000000000000001
80 newTopic 0.04347911384978418 политика 0.01662027870655147  1.0
90 newTopic 0.034655011092361444 роскомнадзор 0.01321823837392982  1.4000000000000001
Wall time: 3min 8s


In [104]:
topicValueResult.head(10) 

,topicName,probValue,countNewWord
0,newTopic,0.133366,2.0
1,newTopic,0.026847,0.8
2,newTopic,0.053038,1.4
3,2014,0.044473,3.8
4,newTopic,0.026554,1.2
5,newTopic,0.041684,1.0
6,newTopic,0.078973,1.8
7,политика,0.016831,0.2
8,newTopic,0.196473,2.2
9,newTopic,0.051747,1.2


In [105]:
topicValueResult['topicName'].value_counts()

newTopic                  71
2014                       5
политика                   5
сша                        1
европа                     1
экскурсии                  1
сергей_помазун             1
спорт                      1
северодвинск               1
ли_юаньчао                 1
дания                      1
россия                     1
верховная_рада_украины     1
ванкувер                   1
блоги                      1
дизайн                     1
манчестер_юнайтед          1
экономика_россии           1
землетрясения              1
женщины                    1
евровидение                1
мариано_рахой              1
Name: topicName, dtype: int64

In [106]:
topicValueResult.groupby(['topicName']).sum()

,probValue,countNewWord
topicName,,
2014,0.206544,6.4
newTopic,4.341900,152.8
блоги,0.066148,0.8
ванкувер,0.062829,0.2
верховная_рада_украины,0.026504,0.8
дания,0.146495,1.6
дизайн,0.062673,0.2
евровидение,0.052229,2.4
европа,0.020074,0.4


In [86]:
print (topicValueResult['countNewWord'].sum(), topicValueResult['countNewWord'].max(), topicValueResult['countNewWord'].min(), topicValueResult['countNewWord'].mean(), topicValueResult['countNewWord'].median())

881 107 1 8.81 6.0
